# For testing the signal generation and filtering techniques

In [ ]:
from NuRadioMC.SignalGen import askaryan as ask
from NuRadioReco.utilities import units, fft
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
plt.style.use('plot_style.txt')

# Compute some signals

In [ ]:
N = 896
dt = 1e-10 * units.second
#theta1 = (55.82 + 19.99)*units.deg
#theta2 = (55.82 - 19.99)*units.deg
theta0 = (55.823359)*units.deg
theta1 = (55.823359 + 3)*units.deg
theta2 = (55.823359 + 6)*units.deg
theta3 = (55.823359 + 10)*units.deg
theta4 = (55.823359 + 15)*units.deg
theta5 = (55.823359 - 3)*units.deg
theta6 = (55.823359 - 6)*units.deg
theta7 = (55.823359 - 10)*units.deg
theta8 = (55.823359 - 15)*units.deg
thetas = [theta0, theta1, theta2, theta3, theta4, theta5, theta6, theta7, theta8]
#theta1 = 55.823359*units.deg
#theta2 = 55.823359*units.deg

sr = 1/dt
ff = np.fft.rfftfreq(N, dt)

traces = []
traces_filtered = []
spectrum = []
spectrum_filtered = []

for theta in thetas:
    trace = ask.get_time_trace(energy=1*units.EeV,
                            theta=theta,
                            N=N,
                            dt=dt,
                            shower_type="had",
                            n_index=1.78,
                            R=1*units.km,
                            model="ARZ2020",
                            iN=1)

    traces.append(trace)
    trace_spectrum = fft.time2freq(trace, sampling_rate=sr)
    spectrum.append(trace_spectrum)
    freqs = np.abs(trace_spectrum)
    i = 0
    while  i+2 < freqs.shape[0] and (not (freqs[i] > freqs[i+1] < freqs[i+2])):
        i = i + 1

    mask = ff < ff[i] #* units.MHz
    
    """ 
    max_freq = lambda x: 5000*np.exp(np.abs(x/units.deg - 56)*(-np.log(5)/10)) * units.MHz
    #max_freq = lambda x: (5000 - np.abs(x/units.deg - 56)*240) * units.MHz
    mask1 = ff < max_freq(theta1)
    mask2 = ff < max_freq(theta2)
     """

    trace_spectrum_filtered = np.zeros((trace_spectrum.shape), dtype=complex)
    trace_spectrum_filtered[mask] = trace_spectrum[mask]
    spectrum_filtered.append(trace_spectrum_filtered)
    trace_filtered = fft.freq2time(trace_spectrum_filtered, sampling_rate=sr)
    traces_filtered.append(trace_filtered)

time = np.arange(0, N)*dt / units.nanosecond

# Plot the filtering of those signals

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(8,5), sharex='col')
ax[0,0].plot(time, traces[3]/units.volt*units.meter*1000, color='#0173b2')
ax[0,0].plot(time, traces[7]/units.volt*units.meter*1000, color='#de8f05')
ax[0,0].set_ylabel('Amplitude [mV/m]')
ax[0,1].plot(ff/units.GHz, np.abs(spectrum[3])*1000, label=r'$\theta$ = +10$^\circ$', color='#0173b2')
ax[0,1].plot(ff/units.GHz, np.abs(spectrum[7])*1000, label=r'$\theta$ = -10$^\circ$', color='#de8f05')
ax[0,1].set_ylabel('Amplitude [mV/m/GHz]')
ax[0,1].legend(frameon=False, loc='upper right')
ax[1,1].plot(ff/units.GHz, np.abs(spectrum_filtered[3])*1000, color='#0173b2')
ax[1,1].plot(ff/units.GHz, np.abs(spectrum_filtered[7])*1000, color='#de8f05')
ax[1,1].set_xlabel('Frequency [GHz]')
ax[1,1].set_ylabel('Amplitude [mV/m/GHz]')
ax[1,0].plot(time, traces_filtered[3]/units.volt*units.meter*1000, color='#0173b2')
ax[1,0].plot(time, traces_filtered[7]/units.volt*units.meter*1000, color='#de8f05')
ax[1,0].set_xlabel('Time [ns]')
ax[1,0].set_ylabel('Amplitude [mV/m]')


ax[0,0].yaxis.set_minor_locator(  AutoMinorLocator(5))
ax[0,0].xaxis.set_minor_locator(  AutoMinorLocator(5))
ax[0,1].yaxis.set_minor_locator(  AutoMinorLocator(5))
ax[0,1].xaxis.set_minor_locator(  AutoMinorLocator(5))
ax[1,0].yaxis.set_minor_locator(  AutoMinorLocator(5))
ax[1,0].xaxis.set_minor_locator(  AutoMinorLocator(5))
ax[1,1].yaxis.set_minor_locator(  AutoMinorLocator(5))
ax[1,1].xaxis.set_minor_locator(  AutoMinorLocator(5))

fig.tight_layout()

#plt.savefig('thesis/Exjobb-rapport/figures/signal_filtering.pdf', dpi=300)

# Plot some example signals

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,5))
colors = ['#ece133', '#de8f05', '#029e73', '#d55e00', '#ece133', '#de8f05', '#029e73', '#d55e00']
deg = r'$^\circ$'
th = r'$\theta$'
handle = []
for i, trace in enumerate(traces_filtered):
    if i == 0:
        ax.plot(time[300:600], trace[300:600]/10  /units.millivolt*units.meter, label=f'{th} = {thetas[i]/units.deg - 55.823359:.2f}{deg}', color='#0173b2')
    elif i <=4:
        ax.plot(time[300:600], trace[300:600]  /units.millivolt*units.meter, label=f'{th} = {thetas[i]/units.deg - 55.823359:.2f}{deg}', color=colors[i-1])
    else:
        ax.plot(time[300:600], trace[300:600]  /units.millivolt*units.meter, '--', color=colors[i-1])

ax.yaxis.set_minor_locator(  AutoMinorLocator(5))
ax.xaxis.set_minor_locator(  AutoMinorLocator(5))

ax.set_xlabel('Time [ns]')
ax.set_ylabel('Amplitude [mV/m]')
#ax.set_ylim((-11,20))

ax.legend(frameon=False, ncol=1, loc='upper right')

plt.text(44.3,10,'0.1x',horizontalalignment='right', color='#0173b2')
#plt.savefig('thesis/Exjobb-rapport/figures/signal_example_neg.pdf', dpi=300)

# Load tensorflow

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
np.set_printoptions(precision=3, suppress=True)

from tensorflow import keras
import numpy as np
np.set_printoptions(precision=3, suppress=True)

from matplotlib import pyplot as plt
import seaborn as sns
pal = sns.color_palette("colorblind")
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

# Define the condition normalization

In [ ]:
latent_dim = 112
N = 896
n_index = 1.78
cherenkov_angle = np.arccos(1. / n_index)

def norm_cond(condition, cherenkov_angle):
    condition_norm = condition.copy()
    condition_norm[:, 0] = (np.log10(condition_norm[:, 0]) - 15)/(19 - 15)
    #condition_norm[:, 1] = ((condition_norm[:, 1] - cherenkov_angle) / units.deg + 2.5)/ 5
    condition_norm[:, 1] = ((condition_norm[:, 1] - cherenkov_angle) / units.deg + 5)/ 10
    return condition_norm

# Load generator and critic models

In [ ]:
name = 'run4-lr=5e-05-critic_filters=24-generator_filters=48-generator_k_size=15'
g_model = keras.models.load_model(f'/mnt/md0/aholmberg/GAN_models/transconv-incept-m14-10deg-05split-fixed/gen_{name}/', compile=False)
g_model.compile()
c_model = keras.models.load_model(f'/mnt/md0/aholmberg/GAN_models/transconv-incept-m14-10deg-05split-fixed/crit_{name}/', compile=False)
c_model.compile()

# Define some condition to use for sigals

In [ ]:
""" condition = np.array([[1e19, cherenkov_angle+5*units.deg],
                      [1e19, cherenkov_angle-5*units.deg],
                      [1e18, cherenkov_angle+5*units.deg],
                      [1e18, cherenkov_angle-5*units.deg],
                      [1e17, cherenkov_angle+5*units.deg],
                      [1e17, cherenkov_angle-5*units.deg],
                      [1e16, cherenkov_angle+5*units.deg],
                      [1e16, cherenkov_angle-5*units.deg],
                      [1e15, cherenkov_angle+5*units.deg],
                      [1e15, cherenkov_angle-5*units.deg]])
 """
condition = np.array([[1e19, cherenkov_angle],
                      [1e19, cherenkov_angle-1*units.deg],
                      [1e19, cherenkov_angle+1*units.deg],
                      [1e19, cherenkov_angle-2*units.deg],
                      [1e19, cherenkov_angle+2*units.deg],
                      [1e19, cherenkov_angle-3*units.deg],
                      [1e19, cherenkov_angle+3*units.deg],
                      [1e19, cherenkov_angle-4*units.deg],
                      [1e19, cherenkov_angle+4*units.deg],
                      [1e19, cherenkov_angle-5*units.deg],
                      [1e19, cherenkov_angle+5*units.deg]])

condition_expanded = np.repeat(condition, 10, axis=0)
cond_norm = norm_cond(condition_expanded, cherenkov_angle)

# Compute true traces

In [ ]:
true_traces = np.zeros((cond_norm.shape[0], N))
for ind in range(cond_norm.shape[0]):
    theta = condition_expanded[ind, 1]
    energy = condition_expanded[ind, 0] * units.eV
    trace = ask.get_time_trace(energy=energy,
                                theta=theta,
                                N=N,
                                dt=dt,
                                shower_type="had",
                                n_index=1.78,
                                R=1*units.km,
                                model="ARZ2020",
                                iN=ind%10)

    trace_spectrum = fft.time2freq(trace, sampling_rate=sr)
    freqs = np.abs(trace_spectrum)
    i = 0
    while  i+2 < freqs.shape[0] and (not (freqs[i] > freqs[i+1] < freqs[i+2])):
        i = i + 1

    mask = ff < ff[i]
    trace_spectrum_filtered = np.zeros((trace_spectrum.shape), dtype=complex)
    trace_spectrum_filtered[mask] = trace_spectrum[mask]
    trace_filtered = fft.freq2time(trace_spectrum_filtered, sampling_rate=sr)
    true_traces[ind, :] = trace_filtered

# Generate and scale fake traces

In [ ]:
latent_vec = tf.random.normal((true_traces.shape[0], latent_dim))
gen_traces = g_model([latent_vec, cond_norm], training=False)

In [ ]:
#gen_traces_scaled = gen_traces / (np.expand_dims(1e19/condition_expanded[:, 0], axis=-1) * np.expand_dims(((condition_expanded[:, 1]/units.deg - cherenkov_angle/units.deg))**4, axis=-1))
gen_traces_scaled = gen_traces / (np.expand_dims(1e19/condition_expanded[:, 0], axis=-1) * (np.expand_dims(((condition_expanded[:, 1]/units.deg - cherenkov_angle/units.deg))**4, axis=-1) + 1)/6) *1000
gen_traces_scaled = gen_traces_scaled.numpy()

# Plot example signals with true bounds

In [ ]:
ind = 9

mean_true_trace = np.zeros((11, 896))
for i in range(11):
    mean_true_trace[i, :] = np.mean(true_traces[i*10:(i+1)*10, :]/units.millivolt, axis=0)

p2p = np.max(true_traces, axis=-1) - np.min(true_traces, axis=-1)

max_trace = np.zeros((11, 896))
min_trace = np.zeros((11, 896))
for i in range(11):  # find the signals with highest/lowest peak to peak amplitude
    tmp = p2p[i*10:(i+1)*10].tolist()
    tmp, index = (list(t) for t in zip(*sorted(zip(tmp, range(10)))))
    min_trace[i, :] = true_traces[i*10 + index[0], :]
    max_trace[i, :] = true_traces[i*10 + index[-1], :]


time = np.arange(0, N)*dt / units.nanosecond
if ind == 0:
    wind = np.arange(430,470)
elif ind == 3:
    wind = np.arange(380,520)
else:
    wind = np.arange(300,600)

theta = r'$\theta$'
pm = r'$\pm$'
fig, ax = plt.subplots(1,1, figsize=(8,5))
if ind == 0:
    ax.fill_between(time[wind], max_trace[ind, wind]/units.millivolt, min_trace[ind, wind]/units.millivolt, facecolor=pal[0], label=f'true signals')
    for n in range(0,4):
        ax.plot(time[wind], gen_traces_scaled[ind*10 + n, wind], color=pal[7], linewidth=1)

    ax.plot(time[wind], gen_traces_scaled[ind*10+9, wind], color=pal[7], linewidth=1, label=f'generated signals')
    name = f'sig_E={int(condition[ind, 0]/units.PeV)}PeV_theta={int(condition[ind, 1]/units.deg - cherenkov_angle/units.deg )}deg.pdf'
else:
    ax.fill_between(time[wind], max_trace[ind, wind]/units.millivolt, min_trace[ind, wind]/units.millivolt, facecolor=pal[0], label=f'true signals -{theta}')
    ax.fill_between(time[wind], max_trace[ind+1, wind]/units.millivolt, min_trace[ind+1, wind]/units.millivolt, facecolor=pal[3], label=f'true signals  {theta}')

    for n in range(0,4):
        ax.plot(time[wind], gen_traces_scaled[ind*10 + n, wind], color=pal[7], linewidth=1)
        ax.plot(time[wind], gen_traces_scaled[(ind+1)*10 + n, wind], color=pal[7], linewidth=1)

    ax.plot(time[wind], gen_traces_scaled[ind*10+9, wind], color=pal[7], linewidth=1, label=f'generated signals {pm}{theta}')
    ax.plot(time[wind], gen_traces_scaled[(ind+1)*10+9, wind], color=pal[7], linewidth=1)
    name = f'sig_E={int(condition[ind, 0]/units.PeV)}PeV_theta={int(condition[ind+1, 1]/units.deg - cherenkov_angle/units.deg )}deg.pdf'

ax.set_xlabel('Time [ns]')
ax.set_ylabel('Amplitude [mV/m]')
ax.yaxis.set_minor_locator(  AutoMinorLocator(5))
ax.xaxis.set_minor_locator(  AutoMinorLocator(5))
ax.legend(frameon=False)

#fig.savefig(f'thesis/Exjobb-rapport/figures/signals/{name}')
print(name)

# Another way of ploting some example signals

In [ ]:
i = 3
fig, ax = plt.subplots(1,1, figsize=(6,5))
mean_true_trace = np.mean(true_traces[i*10:i*10+10, :], axis=0)
time = np.arange(0, N)*dt / units.nanosecond
ax.plot(time, mean_true_trace, 'k', label='mean true signal')
ax.plot(time, gen_traces_scaled[i*10, :], '--', label='generated signal 1')
ax.plot(time, gen_traces_scaled[i*10+1, :], '--', label='generated signal 2')
ax.plot(time, gen_traces_scaled[i*10+2, :], '--', label='generated signal 3')
ax.set_xlabel('Time [ns]')
ax.set_ylabel('Amplitude [V/m]')
fig.tight_layout()
ax.legend()

print(f'Energy: {condition[i, 0]/units.EeV} [EeV], viewing angle: {condition[i, 1]/units.deg:.2f} [$\circ$]')